In [8]:
import pandas as pd
user_rewrite = pd.read_csv("/Users/yeliu/IC/Individual_Project/code/sat3.0/data/user_rewritings.csv", header=None,)

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [17]:
class ParaphraseMachine:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
        self.model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws", force_download=True)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        

    def paraphrase(self, input_sentence, num_of_sentences = 1):
        #encoding = self.tokenizer.encode_plus(input_sentence, padding=True, return_tensors="pt")
        encoding = self.tokenizer(input_sentence, padding=True, return_tensors="pt")
        
        input_ids, attention_masks = encoding["input_ids"].to(self.device), encoding["attention_mask"].to(self.device)


        outputs = self.model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=256,
            do_sample=True,
            top_k=120,
            top_p=0.95,
            early_stopping=True,
            repetition_penalty= 1.5, # adjust this as we see fit to avoid repetition in sentences
            num_return_sequences=num_of_sentences
        )

        result = []
        for i, output in enumerate(outputs):
            output_list = ""
            decoded_sentence = self.tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            output_list = output_list + decoded_sentence + " "
            result.append(output_list)
        return result


In [18]:
pm = ParaphraseMachine()

paraphrase = []
for i in range(len(user_rewrite[0])):
    input_sentence = user_rewrite[0][i]
    generated_sentence = pm.paraphrase(input_sentence, num_of_sentences=1)
    paraphrase.append(generated_sentence)

In [19]:
user_rewrite['paraphrased'] = paraphrase
user_rewrite.columns = ['utterance', 'paraphrased']
user_rewrite.to_csv("/Users/yeliu/IC/Individual_Project/code/sat3.0/data/t5_paraphrased.csv", index=False)